# Xata

>[Xata](https://xata.io) 是一个无服务器数据平台，基于 `PostgreSQL` 和 `Elasticsearch` 构建。它提供了一个 Python SDK 用于与数据库进行交互，并提供一个 UI 用于管理数据。使用 `XataChatMessageHistory` 类，您可以将 Xata 数据库用于聊天会话的长期持久化。

本笔记本将涵盖：

* 一个简单的示例，展示 `XataChatMessageHistory` 的功能。
* 一个更复杂的示例，使用一个 REACT 代理，该代理能够根据知识库或文档（以向量存储的形式存储在 Xata 中）回答问题，并能对其过去的邮件进行长期可搜索的历史记录（以内存存储的形式存储在 Xata 中）

## 设置

### 创建数据库

在 [Xata UI](https://app.xata.io) 中创建一个新数据库。你可以随意命名，在本记事本中，我们将使用 `langchain`。Langchain 集成可以自动创建用于存储记忆的表，这也是我们将在本示例中使用的。如果你想预先创建表，请确保它具有正确的 schema，并在创建类时将 `create_table` 设置为 `False`。预先创建表可以在每次会话初始化时节省一次到数据库的往返。

让我们先安装依赖项：

In [ ]:
%pip install --upgrade --quiet  xata langchain-openai langchain langchain-community

接下来，我们需要获取 Xata 的环境变量。你可以通过访问[账户设置](https://app.xata.io/settings)来创建一个新的 API 密钥。要找到数据库 URL，请前往你已创建数据库的设置页面。数据库 URL 应该看起来像这样：`https://demo-uni3q8.eu-west-1.xata.sh/db/langchain`。

In [ ]:
import getpass

api_key = getpass.getpass("Xata API key: ")
db_url = input("Xata database URL (copy it from your DB settings):")

## 创建一个简单的内存存储

为了单独测试内存存储功能，让我们使用以下代码片段：

In [ ]:
from langchain_community.chat_message_histories import XataChatMessageHistory

history = XataChatMessageHistory(
    session_id="session-1", api_key=api_key, db_url=db_url, table_name="memory"
)

history.add_user_message("hi!")

history.add_ai_message("whats up?")

上述代码创建了一个 ID 为 `session-1` 的会话，并向其中存储了两条消息。运行上述代码后，如果您访问 Xata UI，应该会看到一个名为 `memory` 的表，并且其中添加了两条消息。

您可以使用以下代码检索特定会话的消息历史记录：

In [ ]:
history.messages

## 利用记忆进行的会话式问答链

现在，让我们来看一个更复杂的例子：结合 OpenAI、Xata Vector Store 集成和 Xata Memory Store 集成，创建一个基于您数据的问答聊天机器人，支持追问和历史记录。

我们需要访问 OpenAI API，所以我们来配置 API 密钥：

In [ ]:
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

为了存储聊天机器人将用于搜索答案的文档，请使用 Xata UI 在 `langchain` 数据库中添加一个名为 `docs` 的表，并添加以下列：

* `content` 类型为 "Text"。此列用于存储 `Document.pageContent` 的值。
* `embedding` 类型为 "Vector"。使用您计划使用的模型所使用的维度。在此笔记本中，我们使用的是 OpenAI embeddings，其维度为 1536。

让我们创建向量存储并向其中添加一些示例文档：

In [ ]:
from langchain_community.vectorstores.xata import XataVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

texts = [
    "Xata is a Serverless Data platform based on PostgreSQL",
    "Xata offers a built-in vector type that can be used to store and query vectors",
    "Xata includes similarity search",
]

vector_store = XataVectorStore.from_texts(
    texts, embeddings, api_key=api_key, db_url=db_url, table_name="docs"
)

运行上述命令后，如果您进入 Xata UI，您应该会在 `docs` 表中看到加载的文档及其嵌入。

现在，让我们创建一个 ConversationBufferMemory 来存储来自用户和 AI 的聊天消息。

In [ ]:
from uuid import uuid4

from langchain.memory import ConversationBufferMemory

chat_memory = XataChatMessageHistory(
    session_id=str(uuid4()),  # needs to be unique per user session
    api_key=api_key,
    db_url=db_url,
    table_name="memory",
)
memory = ConversationBufferMemory(
    memory_key="chat_history", chat_memory=chat_memory, return_messages=True
)

现在是时候创建一个 Agent 来同时使用向量存储和聊天记忆了。

In [ ]:
from langchain.agents import AgentType, initialize_agent
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_openai import ChatOpenAI

tool = create_retriever_tool(
    vector_store.as_retriever(),
    "search_docs",
    "Searches and returns documents from the Xata manual. Useful when you need to answer questions about Xata.",
)
tools = [tool]

llm = ChatOpenAI(temperature=0)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)

为了进行测试，我们来告诉代理我们的名字：

In [ ]:
agent.run(input="My name is bob")

现在，让我们问问关于 Xata 的一些问题：

In [ ]:
agent.run(input="What is xata?")

请注意，它会根据存储在文档库中的数据进行回答。现在，让我们提出一个后续问题：

In [ ]:
agent.run(input="Does it support similarity search?")

现在我们来测试一下它的记忆：

In [ ]:
agent.run(input="Did I tell you my name? What is it?")